In [101]:
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns',30)

In [102]:
giay_nam = pd.read_csv('D:\TIKI\giay-nam.csv')
giay_nu = pd.read_csv('D:\TIKI\giay-nu.csv')
giay_nam = giay_nam.drop(columns=['Unnamed: 0'])
giay_nu = giay_nu.drop(columns=['Unnamed: 0'])

In [103]:
giay_nam_id = pd.read_csv('giay-nam-id.csv',usecols=['product_id','category_name'])
giay_nu_id = pd.read_csv('giay-nu-id.csv',usecols=['product_id','category_name'])
giay_nam_id = giay_nam_id.drop_duplicates()
giay_nu_id = giay_nu_id.drop_duplicates()

In [104]:
giay_nam = giay_nam.merge(giay_nam_id,left_on='id',right_on='product_id',how='left')
giay_nu = giay_nu.merge(giay_nu_id,left_on='id',right_on='product_id',how='left')
giay_nam = giay_nam.drop(columns=['product_id'])
giay_nu = giay_nu.drop(columns=['product_id'])

In [105]:
#I add tag 'man' / 'woman' into category_name to ensure that there are no identical category name in both table when concatenated later
giay_nam['category_name'] = 'MAN-'+giay_nam['category_name']
giay_nam['root_category'] = 'giay-nam'

giay_nu['category_name'] = 'WOMAN-'+giay_nu['category_name']
giay_nu['root_category'] = 'giay-nu'

In [106]:
last_update = '2023-06-18 18:00:00'
concat_df = pd.concat([giay_nam,giay_nu])
concat_df['last_update'] = pd.to_datetime(last_update)
concat_df = concat_df.reset_index()
concat_df = concat_df.drop(columns='index')

In [107]:
concat_df.sample(3)

,id,sku,name,discription,original_price,list_price,price,alltime_quantity_sold,quantity_sold,inventory_status,fulfillment_type,brand,category_id,category,review_count,rating_average,favourite_count,ASA_cashback,pay_later,current_seller,date_created,video_url,category_name,root_category,last_update
18962,216416795,3598612806613,Hot 2023 Guốc nữ thời trang đẳng cấp quí phái ...,"MÔ TẢ SẢN PHẨM Ra đời tầm hè 2020, nố...",550000,550000,550000,{},NaN,available,dropship,OEM,2,Root,0,0.0,0,"Thưởng 2,75 ASA (≈ 588đ)","{'title': 'Trả góp 0%', 'summary': 'Mua trước ...",Hoa Lan TV,135,NaN,WOMAN-dep-quai-ngang,giay-nu,2023-06-18 18:00:00
20733,38214285,6812461864114,Giày thể thao PTTT-0130,Giày thể thao PTTT-0130 với thiết kế trẻ trung...,228000,228000,228000,{},NaN,available,dropship,OEM,2,Root,0,0.0,0,"Thưởng 1,14 ASA (≈ 245đ)","{'title': 'Trả góp 0%', 'summary': 'Mua trước ...",GIẦY DÉP PHONG THÚY,1328,NaN,WOMAN-giay-the-thao-co-thap,giay-nu,2023-06-18 18:00:00
39236,141124111,3455975336062,Giày búp bê nữ BIGGBEN da bò cao cấp BB114,Chất Liệu :Da bò thật 100%Kích Thước :từ 35 đế...,350000,350000,249000,4,4.0,available,dropship,BIGGBEN,49566,Giày búp bê mũi tròn,0,0.0,0,"Thưởng 1,25 ASA (≈ 266đ)","{'title': 'Trả góp 0%', 'summary': 'Mua trước ...",Shoes BIGBEN,578,NaN,WOMAN-giay-bup-be-mui-tron,giay-nu,2023-06-18 18:00:00


## PREPROCESSING DATA

In [108]:
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42981 entries, 0 to 42980
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     42981 non-null  int64         
 1   sku                    42981 non-null  int64         
 2   name                   42981 non-null  object        
 3   discription            42981 non-null  object        
 4   original_price         42981 non-null  int64         
 5   list_price             42981 non-null  int64         
 6   price                  42981 non-null  int64         
 7   alltime_quantity_sold  42981 non-null  object        
 8   quantity_sold          13504 non-null  float64       
 9   inventory_status       42981 non-null  object        
 10  fulfillment_type       42980 non-null  object        
 11  brand                  42981 non-null  object        
 12  category_id            42981 non-null  int64         
 13  c

In [109]:
concat_df['has_video'] = concat_df['video_url'].notna()
concat_df['alltime_quantity_sold'] = concat_df['alltime_quantity_sold'].replace('{}',0)
concat_df['quantity_sold'] = concat_df['quantity_sold'].fillna(0)
concat_df['rating_average'] = concat_df['rating_average']
concat_df['ASA_cashback_transformed'] = concat_df['ASA_cashback'].apply(lambda x: float(x.split()[1].replace(',','.')))
concat_df['pay_later'] = concat_df['pay_later'].notna()

# check duplicate columns

In [116]:
#check 'alltime_quantity_sold' vs 'quantity_sold'
print(concat_df.loc[concat_df['alltime_quantity_sold'].astype(float) != concat_df['quantity_sold']])

#check 'original_price' vs 'list_price'
print(concat_df.loc[concat_df['list_price'] != concat_df['original_price']])

Empty DataFrame
Columns: [id, sku, name, discription, original_price, list_price, price, alltime_quantity_sold, quantity_sold, inventory_status, fulfillment_type, brand, category_id, category, review_count, rating_average, favourite_count, ASA_cashback, pay_later, current_seller, date_created, video_url, category_name, root_category, last_update, has_video, ASA_cashback_transformed]
Index: []
Empty DataFrame
Columns: [id, sku, name, discription, original_price, list_price, price, alltime_quantity_sold, quantity_sold, inventory_status, fulfillment_type, brand, category_id, category, review_count, rating_average, favourite_count, ASA_cashback, pay_later, current_seller, date_created, video_url, category_name, root_category, last_update, has_video, ASA_cashback_transformed]
Index: []


In [118]:
concat_df = concat_df.drop(columns=['quantity_sold','original_price'])

## DATA VISUALIZATIONS 